IMPORTING OF LIBRARIES


---



In [1]:
import requests, json
from lxml import html
from bs4 import BeautifulSoup
import time

DATA ACCESS THROUGH API


---



The HealthIT.Gov webpage is accessed, where web scraping and web crawling where carried out to obtain the list of datasets available, and to grab the URL extensions for the same. The fetched extensions were used to navigate to that website, where again, the data's API URL was obtained, following which the data is gathered.

In [2]:
url = "https://www.healthit.gov/data/api"

response1 = requests.get(url)
data = response1.content
source_code = html.fromstring(data)

print("The list of Databases in the HealthIT.gov Website: \n")
for i in range(1,24):
  path = f"/html/body/div[1]/div/main/section/div/div[2]/article/div/div/div/table/tbody/tr[{i}]/td[1]"
  db = source_code.xpath(path)
  print(db[0].text_content())

The list of Databases in the HealthIT.gov Website: 

2015 Edition Market Readiness for Hospitals and Clinicians Data
Centers for Medicare & Medicaid Services (CMS) EHR Incentive Program Measures
EHR Developers Reported by Health Care Providers Participating in Federal Programs
EHR Products Used for Meaningful Use Attestation
Effects of Meaningful Use Functionalities on Health Care Quality, Safety, and Efficiency
Electronic Prescribing Adoption and Use by County
Electronic Prescribing Adoption and Use by State
Federal Health IT Strategic Plan: 2015-2020 Goals
Hospital Public Health Reporting
Non-federal Acute Care Hospital Health IT Adoption and Use
ONC Budget Performance Measure Data
ONC Community College Consortia to Educate Health IT Professionals Program Key Performance Indicators (KPIs)
ONC Health Information Technology for Economic and Clinical Health (HITECH) Grantee Crosswalk
ONC Health Information Technology for Economic and Clinical Health (HITECH) Grantee List
ONC Regional Ex

In [3]:
soup = BeautifulSoup(response1.text, 'html.parser')
#print(soup)
datasets = []
for link in soup.find_all('a'):
  path = link.get('href')
  if "data" in path:
    datasets.append(path)
datasets = datasets[4:]
print(datasets)

['/data/datasets/2015-edition-market-readiness-hospitals-and-clinicians-data', '/data/datasets/centers-medicare-medicaid-services-cms-ehr-incentive-program-measures', '/data/datasets/ehr-developers-reported-health-care-providers-participating-federal-programs', '/data/datasets/ehr-products-used-meaningful-use-attestation', '/data/datasets/effects-meaningful-use-functionalities-health-care-quality-safety-and-efficiency', '/data/datasets/electronic-prescribing-adoption-and-use-county', '/data/datasets/electronic-prescribing-adoption-and-use-state', '/data/datasets/federal-health-it-strategic-plan-2015-2020-goals', '/data/datasets/hospital-public-health-reporting', '/data/datasets/non-federal-acute-care-hospital-health-it-adoption-and-use', '/data/datasets/onc-budget-performance-measure-data', '/data/datasets/onc-community-college-consortia-educate-health-it-professionals-program-key', '/data/datasets/onc-health-information-technology-economic-and-clinical-health-hitech-grantee', '/data/d

In [4]:
dataset_url = datasets[5]
data_url = url[:len(url) - 9] + dataset_url
print(data_url)

https://www.healthit.gov/data/datasets/electronic-prescribing-adoption-and-use-county


In [5]:
dataset_url2 = datasets[-6]
data_url2 = url[:len(url) - 9] + dataset_url2
print(data_url2)

https://www.healthit.gov/data/datasets/office-based-health-care-providers-database


In [6]:
response2 = requests.get(data_url)
soup2 = BeautifulSoup(response2.text, 'html.parser')
#print(soup2)
final_url = []
for link in soup2.find_all('a'):
  path = link.get('href')
  if "api" in path:
    final_url.append(path)

In [7]:
response3 = requests.get(data_url2)
soup3 = BeautifulSoup(response3.text, 'html.parser')
#print(soup2)
final_url2 = []
for link in soup3.find_all('a'):
  path = link.get('href')
  if "api" in path:
    final_url2.append(path)

In [8]:
data_1 = requests.get(final_url[0]).json()
data_1[0] #[-1] is none region

{'County FIPS': '1',
 'County Name': 'Autauga',
 'FIPS': '1001',
 'Percent of Physicians Actively Using an EHR to e-Prescribe via the Surescripts Network': '0.33',
 'Period': '2014-04',
 'Region': 'Alabama',
 'Region Code': 'AL',
 'State FIPS': '1'}

In [9]:
data_2 = requests.get(final_url2[0]).json()
data_2[0]

{'all_nurse_practitioners': '68473',
 'all_physician_assistants': '49776',
 'all_physicians': '598352',
 'all_primary_care_nurse_practitioners': '41750',
 'all_primary_care_physician_assistants': '22633',
 'all_primary_care_physicians': '238356',
 'all_primary_care_providers': '302739',
 'all_providers': '716594',
 'county_fips': '',
 'fips': '',
 'period': '2011',
 'region': 'National',
 'region_code': 'US',
 'state_fips': '0'}

In [10]:
data_1.pop()
data_2.pop()

{'region': None}

COMBINING THE DATASETS


---



The datasets obtained from both the websited where combined together by matching the FIPS of each county in the data. The healthcare professional's information from the second data is added to the first dataset. Then, the data was converted into a list of list with dictionaries where each sublist denotes a specific state and each dictionary representing a county in them.

In [11]:
for i in data_2:
  for j in data_1:
    if i["period"] == "2013" and i["fips"] == j["FIPS"]:
      j["all_providers"] = i["all_providers"]
      j["all_physicians"] = i["all_physicians"]
      j["all_nurse_practitioners"] = i["all_nurse_practitioners"]
      j["all_physician_assistants"] = i["all_physician_assistants"]
      j["all_primary_care_providers"] = i["all_primary_care_providers"]
      j["all_primary_care_physicians"] = i["all_primary_care_physicians"]
      j["all_primary_care_nurse_practitioners"] = i["all_primary_care_nurse_practitioners"]
      j["all_primary_care_physician_assistants"] = i["all_primary_care_physician_assistants"]
for j in data_1:
  if j["FIPS"] == "51640" or j["FIPS"] == "51678":
      j["all_providers"] = "0"
      j["all_physicians"] = "0"
      j["all_nurse_practitioners"] = "0"
      j["all_physician_assistants"] = "0" 
      j["all_primary_care_providers"] = "0"
      j["all_primary_care_physicians"] = "0"
      j["all_primary_care_nurse_practitioners"] = "0"
      j["all_primary_care_physician_assistants"] = "0"

In [12]:
states = []
for i in data_1:
  states.append(i["Region"])
states_list = list(set(states))
states_list = sorted(states_list)
#print(states_list)

In [13]:
region = ['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado', 'Connecticut', 'Delaware', 'District Of Columbia', 'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey', 'New Mexico', 'New York', 'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania', 'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington', 'West Virginia', 'Wisconsin', 'Wyoming']

In [14]:
temp = []
for i in states_list:
  temp_list = []
  for j in data_1:
    if i == j["Region"]:
      temp_list.append(j)
  temp.append(temp_list)

In [15]:
temp[0][0:2]

[{'County FIPS': '1',
  'County Name': 'Autauga',
  'FIPS': '1001',
  'Percent of Physicians Actively Using an EHR to e-Prescribe via the Surescripts Network': '0.33',
  'Period': '2014-04',
  'Region': 'Alabama',
  'Region Code': 'AL',
  'State FIPS': '1',
  'all_nurse_practitioners': '24',
  'all_physician_assistants': '8',
  'all_physicians': '60',
  'all_primary_care_nurse_practitioners': '10',
  'all_primary_care_physician_assistants': '4',
  'all_primary_care_physicians': '29',
  'all_primary_care_providers': '43',
  'all_providers': '91'},
 {'County FIPS': '3',
  'County Name': 'Baldwin',
  'FIPS': '1003',
  'Percent of Physicians Actively Using an EHR to e-Prescribe via the Surescripts Network': '0.68',
  'Period': '2014-04',
  'Region': 'Alabama',
  'Region Code': 'AL',
  'State FIPS': '1',
  'all_nurse_practitioners': '38',
  'all_physician_assistants': '18',
  'all_physicians': '275',
  'all_primary_care_nurse_practitioners': '24',
  'all_primary_care_physician_assistants': 

TREE DATA STRUCTURE AND REPRESENTATION


---



The anytree package is utilised which assists in creating a tree data structure of the data. Here the root or foundational node is USA, where the different states are its children nodes, and finally the counties within these states are its children nodes. Thus, this is a three layered or level dataset. Each county has some attributes and values namely FIPS, adoption percentage and so on.

In [16]:
pip install anytree

In [17]:
from anytree import Node, RenderTree, AsciiStyle

In [18]:
root1 = Node("USA")
for i in range(len(states_list)):
  states_list[i] = Node(states_list[i], parent = root1)
  for j in range(len(temp)):
    for k in range(len(temp[j])):
      if i == j:
        temp[j][k] = Node(temp[j][k]["County Name"], parent = states_list[i],
                          County_FPS = temp[j][k]["FIPS"],
                          Providers = temp[j][k]["all_providers"],
                          Physicians = temp[j][k]["all_physicians"],
                          Nurse = temp[j][k]["all_nurse_practitioners"],
                          Assistants = temp[j][k]["all_physician_assistants"],
                          Primary_Providers = temp[j][k]["all_primary_care_providers"],
                          Primary_Physicians = temp[j][k]["all_primary_care_physicians"],
                          Primary_Nurse = temp[j][k]["all_primary_care_nurse_practitioners"],
                          Primary_Assistants = temp[j][k]["all_primary_care_physician_assistants"],
                          Percentage = temp[j][k]["Percent of Physicians Actively Using an EHR to e-Prescribe via the Surescripts Network"])

In [19]:
for pre, fill, node in RenderTree(root1):
  print("%s%s" % (pre, node.name))

USA
├── Alabama
│   ├── Autauga
│   ├── Baldwin
│   ├── Barbour
│   ├── Bibb
│   ├── Blount
│   ├── Bullock
│   ├── Butler
│   ├── Calhoun
│   ├── Chambers
│   ├── Cherokee
│   ├── Chilton
│   ├── Choctaw
│   ├── Clarke
│   ├── Clay
│   ├── Cleburne
│   ├── Coffee
│   ├── Colbert
│   ├── Conecuh
│   ├── Coosa
│   ├── Covington
│   ├── Crenshaw
│   ├── Cullman
│   ├── Dale
│   ├── Dallas
│   ├── DeKalb
│   ├── Elmore
│   ├── Escambia
│   ├── Etowah
│   ├── Fayette
│   ├── Franklin
│   ├── Geneva
│   ├── Greene
│   ├── Hale
│   ├── Henry
│   ├── Houston
│   ├── Jackson
│   ├── Jefferson
│   ├── Lamar
│   ├── Lauderdale
│   ├── Lawrence
│   ├── Lee
│   ├── Limestone
│   ├── Lowndes
│   ├── Macon
│   ├── Madison
│   ├── Marengo
│   ├── Marion
│   ├── Marshall
│   ├── Mobile
│   ├── Monroe
│   ├── Montgomery
│   ├── Morgan
│   ├── Perry
│   ├── Pickens
│   ├── Pike
│   ├── Randolph
│   ├── Russell
│   ├── Saint Clair
│   ├── Shelby
│   ├── Sumter
│   ├── Talladega
│   ├── Tallapoosa
│   ├──

In [20]:
from anytree.exporter import DotExporter
import graphviz

In [21]:
print(root1.children)

(Node('/USA/Alabama'), Node('/USA/Alaska'), Node('/USA/Arizona'), Node('/USA/Arkansas'), Node('/USA/California'), Node('/USA/Colorado'), Node('/USA/Connecticut'), Node('/USA/Delaware'), Node('/USA/District Of Columbia'), Node('/USA/Florida'), Node('/USA/Georgia'), Node('/USA/Hawaii'), Node('/USA/Idaho'), Node('/USA/Illinois'), Node('/USA/Indiana'), Node('/USA/Iowa'), Node('/USA/Kansas'), Node('/USA/Kentucky'), Node('/USA/Louisiana'), Node('/USA/Maine'), Node('/USA/Maryland'), Node('/USA/Massachusetts'), Node('/USA/Michigan'), Node('/USA/Minnesota'), Node('/USA/Mississippi'), Node('/USA/Missouri'), Node('/USA/Montana'), Node('/USA/Nebraska'), Node('/USA/Nevada'), Node('/USA/New Hampshire'), Node('/USA/New Jersey'), Node('/USA/New Mexico'), Node('/USA/New York'), Node('/USA/North Carolina'), Node('/USA/North Dakota'), Node('/USA/Ohio'), Node('/USA/Oklahoma'), Node('/USA/Oregon'), Node('/USA/Pennsylvania'), Node('/USA/Rhode Island'), Node('/USA/South Carolina'), Node('/USA/South Dakota'),

In [22]:
DotExporter(root1.children[2]).to_picture("Sample.png") # A sample of the tree for a state

Code to create image representation of all the states and their respective counties.

In [23]:
for i in root1.children:
  filename = i.name+"_tree.png"
  print(filename)
  DotExporter(i).to_picture(filename)

Alabama_tree.png
Alaska_tree.png
Arizona_tree.png
Arkansas_tree.png
California_tree.png
Colorado_tree.png
Connecticut_tree.png
Delaware_tree.png
District Of Columbia_tree.png
Florida_tree.png
Georgia_tree.png
Hawaii_tree.png
Idaho_tree.png
Illinois_tree.png
Indiana_tree.png
Iowa_tree.png
Kansas_tree.png
Kentucky_tree.png
Louisiana_tree.png
Maine_tree.png
Maryland_tree.png
Massachusetts_tree.png
Michigan_tree.png
Minnesota_tree.png
Mississippi_tree.png
Missouri_tree.png
Montana_tree.png
Nebraska_tree.png
Nevada_tree.png
New Hampshire_tree.png
New Jersey_tree.png
New Mexico_tree.png
New York_tree.png
North Carolina_tree.png
North Dakota_tree.png
Ohio_tree.png
Oklahoma_tree.png
Oregon_tree.png
Pennsylvania_tree.png
Rhode Island_tree.png
South Carolina_tree.png
South Dakota_tree.png
Tennessee_tree.png
Texas_tree.png
Utah_tree.png
Vermont_tree.png
Virginia_tree.png
Washington_tree.png
West Virginia_tree.png
Wisconsin_tree.png
Wyoming_tree.png


SEARCHING THE TREE


---

The following code snippets showcase the usage of search methods to traverse through the tree to find the specified node. The level of searching can be controlled. It is very useful as there are instances of regions have same names at different locations and levels.

In [24]:
from anytree.search import findall_by_attr

In [ ]:
print(RenderTree(root1, style=AsciiStyle()).by_attr())

In [26]:
findall_by_attr(root1, "Wyoming")  #[-1].children

(Node('/USA/New York/Wyoming', Assistants='11', County_FPS='36121', Nurse='13', Percentage='0.89', Physicians='34', Primary_Assistants='4', Primary_Nurse='10', Primary_Physicians='17', Primary_Providers='31', Providers='58'),
 Node('/USA/Pennsylvania/Wyoming', Assistants='10', County_FPS='42131', Nurse='5', Percentage='0.69', Physicians='30', Primary_Assistants='10', Primary_Nurse='5', Primary_Physicians='23', Primary_Providers='38', Providers='45'),
 Node('/USA/West Virginia/Wyoming', Assistants='12', County_FPS='54109', Nurse='3', Percentage='0.68', Physicians='6', Primary_Assistants='12', Primary_Nurse='3', Primary_Physicians='5', Primary_Providers='20', Providers='21'),
 Node('/USA/Wyoming'))

In [27]:
findall_by_attr(root1, "Platte")

(Node('/USA/Missouri/Platte', Assistants='4', County_FPS='29165', Nurse='16', Percentage='0.68', Physicians='104', Primary_Assistants='2', Primary_Nurse='10', Primary_Physicians='49', Primary_Providers='61', Providers='124'),
 Node('/USA/Nebraska/Platte', Assistants='5', County_FPS='31141', Nurse='9', Percentage='0.87', Physicians='38', Primary_Assistants='3', Primary_Nurse='8', Primary_Physicians='20', Primary_Providers='31', Providers='52'),
 Node('/USA/Wyoming/Platte', Assistants='1', County_FPS='56031', Nurse='4', Percentage='0.33', Physicians='16', Primary_Assistants='1', Primary_Nurse='4', Primary_Physicians='5', Primary_Providers='10', Providers='21'))

Using level control in the search function to limit and correctly obtain the required location from both the state and county name.

In [28]:
findall_by_attr(findall_by_attr(root1, "Wyoming", maxlevel=2)[0], "Platte")[0]

Node('/USA/Wyoming/Platte', Assistants='1', County_FPS='56031', Nurse='4', Percentage='0.33', Physicians='16', Primary_Assistants='1', Primary_Nurse='4', Primary_Physicians='5', Primary_Providers='10', Providers='21')

INTERACTION


---

The following code chunks aims to gather user input for state name and county name. First, the tree representation for the state will be created. These will be used to obtain the county with the lowest adoption rate in the given state. Then, details about the county will be listed out. 

In [51]:
state_name = input("Enter the name of the State = ")
if state_name not in region:
  print("Invalid State Name")

Enter the name of the State = Michigan


In [30]:
county_name = input("Enter the name of the County = ")

Enter the name of the County = Washtenaw


In [31]:
findall_by_attr(root1, state_name, maxlevel=2)[0]

Node('/USA/Michigan')

In [32]:
findall_by_attr(findall_by_attr(root1, state_name, maxlevel=2)[0], county_name)[0]

Node('/USA/Michigan/Washtenaw', Assistants='89', County_FPS='26161', Nurse='122', Percentage='1.00', Physicians='1688', Primary_Assistants='18', Primary_Nurse='56', Primary_Physicians='449', Primary_Providers='523', Providers='1899')

In [33]:
filename = state_name+"_tree.png"
print(filename)
for i in root1.children:
  if i.name == state_name:
    DotExporter(i).to_picture(filename)

Michigan_tree.png


In [34]:
info = list(findall_by_attr(root1, state_name, maxlevel=2))
print("Total Number of Counties in the State = ", len(info[0].children))
perc1 = []
for i in info[0].children:
  perc1.append(float(i.Percentage))
for i in range(len(info[0].children)):
  for j in range(len(perc1)):
    if i == j and float(info[0].children[i].Percentage) == min(perc1):
        print(f"County with the Lowest Percentage of {info[0].children[i].Percentage} is {info[0].children[i].name} [{info[0].children[i].County_FPS}].")


Total Number of Counties in the State =  81
County with the Lowest Percentage of 0.00 is Luce [26095].


In [59]:
print("Checking for State's Care Provider count, Physician Count, Nurse Count, or Assistant Count")
print("Press 1 for Care Provider")
print("Press 2 for Physician Count")
print("Press 3 for Nurse Count")
print("Press 4 for Assistant Count")
check = input("Press 1, 2, 3, or 4 accordingly = ")
if check not in ["1", "2", "3", "4"]:
  print("Invalid Option")

Checking for State's Care Provider count, Physician Count, Nurse Count, or Assistant Count
Press 1 for Care Provider
Press 2 for Physician Count
Press 3 for Nurse Count
Press 4 for Assistant Count
Press 1, 2, 3, or 4 accordingly = 4


In [60]:
if check == "1":
  prov = []
  for i in info[0].children:
    prov.append(int(i.Providers))
  avg = sum(prov) // len(prov)
  print(f"Counties in {state_name} having Care Providers above the average count: \n")
  for i in range(len(info[0].children)):
    for j in range(len(prov)):
      if i == j and int(info[0].children[i].Providers) >= avg:
        print(f"{info[0].children[i].name} with {info[0].children[i].Providers} providers")

if check == "2":
  prov = []
  for i in info[0].children:
    prov.append(int(i.Physicians))
  avg = sum(prov) // len(prov)
  print(f"Counties in {state_name} having Physicians above the average count: \n")
  for i in range(len(info[0].children)):
    for j in range(len(prov)):
      if i == j and int(info[0].children[i].Physicians) >= avg:
        print(f"{info[0].children[i].name} with {info[0].children[i].Physicians} Physicians")

if check == "3":
  prov = []
  for i in info[0].children:
    prov.append(int(i.Nurse))
  avg = sum(prov) // len(prov)
  print(f"Counties in {state_name} having Nurse above the average count: \n")
  for i in range(len(info[0].children)):
    for j in range(len(prov)):
      if i == j and int(info[0].children[i].Nurse) >= avg:
        print(f"{info[0].children[i].name} with {info[0].children[i].Nurse} Nurses")

if check == "4":
  prov = []
  for i in info[0].children:
    prov.append(int(i.Assistants))
  avg = sum(prov) // len(prov)
  print(f"Counties in {state_name} having Assistants above the average count: \n")
  for i in range(len(info[0].children)):
    for j in range(len(prov)):
      if i == j and int(info[0].children[i].Assistants) >= avg:
        print(f"{info[0].children[i].name} with {info[0].children[i].Assistants} Assistants")

Counties in Michigan having Assistants above the average count: 

Berrien with 40 Assistants
Calhoun with 49 Assistants
Genesee with 75 Assistants
Grand Traverse with 37 Assistants
Ingham with 100 Assistants
Jackson with 26 Assistants
Kalamazoo with 122 Assistants
Kent with 255 Assistants
Macomb with 112 Assistants
Marquette with 33 Assistants
Montcalm with 38 Assistants
Muskegon with 59 Assistants
Oakland with 197 Assistants
Ottawa with 74 Assistants
Saginaw with 39 Assistants
Saint Clair with 26 Assistants
Washtenaw with 89 Assistants
Wayne with 174 Assistants


In [ ]:
pip install fastcache

In [38]:
from fastcache import clru_cache as _cache
from anytree import search

In [39]:
@_cache(64)
def cached_search(node, value, name="name", maxlevel=None, mincount=None, maxcount=None):
    return search.findall_by_attr(node, value, name=name, maxlevel=maxlevel, mincount=mincount, maxcount=maxcount)

In [40]:
cache_data = cached_search(cached_search(root1, state_name, maxlevel=2)[0], county_name)[0]

In [41]:
print("County Name = ", cache_data.name)
print("Adoption Percentage = ", cache_data.Percentage)
print("Number of Physicians = ", cache_data.Physicians)
print("Number of Assistants = ", cache_data.Assistants)
print("Number of Nurses = ", cache_data.Nurse)
print("Number of Care Providers = ", cache_data.Providers)

County Name =  Washtenaw
Adoption Percentage =  1.00
Number of Physicians =  1688
Number of Assistants =  89
Number of Nurses =  122
Number of Care Providers =  1899


Code to visualise the tree in a JSON file. The DOT file was generated for the tree, which was cleaned by converting it into a pandas dataframe before converting it into the json format.

In [ ]:
import pandas as pd
df = pd.read_csv("tree_test.txt", sep="->", names=["Parent", "Child"])
df["Child"] = df["Child"].apply(lambda x: x[:len(x) - 1])
df["Child"] = df["Child"].apply(lambda x: x.strip())
df["Parent"] = df["Parent"].apply(lambda x: x.strip())
df["Child"] = df["Child"].apply(lambda x: x[1:-1])
df["Parent"] = df["Parent"].apply(lambda x: x[1:-1])
tree_json = df.values.tolist()

In [44]:
country = {}
for i in tree_json:
  if i[0] == "USA":
    country[i[1]] = []
for i in tree_json:
  for j in states_list:
    if i[0] == j.name:
      country[i[0]].append(i[1])
country_1 = {}
country_1["USA"] = country
country_json = json.dumps(country_1, indent = 4)
#country_json

In [45]:
with open('tree.json', 'w') as outfile:
    outfile.write(country_json)